In [2]:
import pandas as pd
import json

# info

In [ ]:
docker compose up -d
http://localhost:8080/console
psql -h localhost -p 5432 -U postgres -d postgres   

# Create data

In [ ]:
import json
import pandas as pd

# Load your data
with open('protein_id_annotation_position.json', 'r') as f:
    protein_id_annotation_position = json.load(f)

rows = []

# From protein_id_annotation_position.json (using actual annotation)
for protein_id, annotations in protein_id_annotation_position.items():
    for annotation, positions in annotations.items():
        # Determine annotation_type
        if annotation.startswith('['):
            annotation_type = 'PTM'
        elif annotation in ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']:
            annotation_type = 'AA'
        elif annotation in ['IDR', 'PPII-helix', 'bend', 'turn', 'unassigned', 'ß-bridge', 'ß-strand', 'α-helix', '3₁₀-helix', 'π-helix', 'loop']:
            annotation_type = 'sec'
        elif annotation.startswith('IPR'):
            annotation_type = 'domain'
        for position in positions:
            rows.append({
                'protein_id': protein_id,
                'position': str(position),
                'annotation': annotation,
                'annotation_type': annotation_type
            })

# Optional: Deduplicate
df = pd.DataFrame(rows)
df = df.drop_duplicates(subset=['protein_id', 'position', 'annotation', 'annotation_type'])

# Export to CSV
df.to_csv('protmodcon.csv', index=False)

In [2]:
import csv
import psycopg2
from psycopg2.extras import execute_values
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Update these with your actual credentials
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "postgrespassword"
DB_HOST = "localhost"
CSV_FILE = "protmodcon.csv"  # Path to your CSV file

def load_csv_to_postgres(csv_file):
    # Read CSV
    with open(csv_file, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        rows = [
            (row['protein_id'], row['position'], row['annotation'], row['annotation_type'])
            for row in reader
        ]

    logger.info(f"Read {len(rows)} rows from {csv_file}")

    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST
    )
    cur = conn.cursor()

    # Bulk insert
    query = """
        INSERT INTO protmodcon (protein_id, position, annotation, annotation_type)
        VALUES %s
        ON CONFLICT (protein_id, position, annotation) DO NOTHING
    """
    execute_values(cur, query, rows, page_size=1000)
    conn.commit()
    cur.close()
    conn.close()
    logger.info(f"Inserted {len(rows)} rows into PostgreSQL table protmodcon.")

if __name__ == "__main__":
    load_csv_to_postgres(CSV_FILE)

2025-05-28 14:26:55,617 - INFO - Read 28210695 rows from protmodcon.csv
2025-05-28 14:37:21,425 - INFO - Inserted 28210695 rows into PostgreSQL table protmodcon.


# Parse database

In [1]:
import requests

HASURA_URL = "http://localhost:8080/v1/graphql"

def fetch_protein_sequence(protein_id, annotation_type):
    query = '''
    query ($protein_id: String!, $annotation_type: String!) {
      protmodcon(
        where: {
          protein_id: { _eq: $protein_id },
          annotation_type: { _eq: $annotation_type }
        }
      ) {
        annotation
      }
    }
    '''
    variables = {
        "protein_id": protein_id,
        "annotation_type": annotation_type
    }
    headers = {"Content-Type": "application/json"}
    response = requests.post(
        HASURA_URL,
        json={"query": query, "variables": variables},
        headers=headers
    )
    data = response.json()
    # Extract and concatenate all annotation values
    annotations = [item["annotation"] for item in data["data"]["protmodcon"]]
    sequence = ''.join(annotations)
    return sequence

# Example usage:
protein_sequence = fetch_protein_sequence("P41227", "AA")
print(protein_sequence)


MNIRNARPEDLMNMQHCNLLCLPENYQMKYYFYHGLSWPQLSYIAEDENGKIVGYVLAKMEEDPDDVPHGHITSLAVKRSHRRLGLAQKLMDQASRAMIENFNAKYVSLHVRKSNRAALHLYSNTLNFQISEVEPKYYADGEDAYAMKRDLTQMADELRRHLELKEKGRHVVLGAIENKVESKGNSPPSSGEACREEKGLAAEDSGGDSKDLSEVSETTESTDVKDSSEASDSAS


# FASTA

In [9]:
from Bio import SeqIO

def get_protein_ids(fasta_file):
    ids = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        ids.append(record.id)
    return ids

# Usage
protein_ids = get_protein_ids("/Users/woutdemuer/Downloads/uniprotkb_organism_id_9606_AND_cc_scl_n_2025_06_04.fasta")

# Extract the part between the first and second '|'
proteins = [item.split('|')[1] for item in protein_ids]

print(' '.join(proteins))

# ER proteins -> expect enrichment of glycsoylation

O00219 O00526 O14656 O15027 O15547 O43323 O43889 O75631 O95406 O96008 P00533 P06340 P07996 P09038 P09958 P10909 P11801 P13569 P13765 P15515 P16157 P16615 P17861 P18850 P20309 P27824 P29590 P31415 P35638 P40967 P45844 P49662 P51572 P51610 P52292 P55145 P57088 P61619 Q14623 Q15041 Q15465 Q15738 Q3KR37 Q53YD1 Q5BJD5 Q5SNT2 Q5T7P8 Q5VTY9 Q68CJ9 Q6UX53 Q6UXZ3 Q70SY1 Q7Z403 Q7Z4N2 Q86TL2 Q8IU68 Q8IUY3 Q8IV08 Q8IXL6 Q8N6F1 Q8NHH9 Q8NHP6 Q8TEY5 Q8WUJ3 Q92575 Q92819 Q95460 Q969F0 Q96A70 Q96B96 Q96BA8 Q96BY9 Q96CP6 Q96D31 Q96G97 Q96LJ7 Q96N66 Q96S66 Q99941 Q9BT56 Q9BUR5 Q9C0E8 Q9H295 Q9H8H3 Q9H8Y8 Q9H9Z2 Q9HB20 Q9NP72 Q9NP78 Q9NQC3 Q9NR96 Q9NTJ5 Q9NV29 Q9P1W3 Q9UBM1 Q9UBS4 Q9UMX0 Q9UPI3 Q9Y5I7 Q9Y6D0 Q9Y6I9 Q9Y6M5 Q8IWB9 Q8IYS0 Q9BT76


# NEW PROTMODCON

In [5]:
data = {'A0A024RBG1': {'IPR000086': [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145], 'IPR047198': [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144], 'A': [21, 22, 62, 63, 71, 126, 138, 151, 165, 170, 171], 'C': [23, 25, 132, 148], 'D': [13, 32, 44, 89, 107, 110, 125, 162], 'E': [15, 29, 31, 33, 54, 56, 57, 58, 66, 69, 70, 85, 103, 106, 109, 119, 124, 139, 142], 'F': [4, 17, 26, 84, 121, 167], 'G': [16, 50, 51, 52, 60, 61, 72, 75, 78, 82, 115, 147, 153, 175], 'H': [92, 133, 137], 'I': [47, 83, 104, 114, 127], 'IDR': [1, 2, 3, 151, 152, 154, 155, 156, 157, 158, 163, 164, 165, 166, 167, 168, 169, 170, 174, 175, 176, 177, 178, 179, 180, 181], 'K': [3, 5, 18, 19, 74, 76, 91, 117, 122, 128, 134, 143, 145], 'L': [24, 35, 36, 77, 80, 81, 99, 105, 130, 141, 144, 146, 160, 166, 176], 'M': [1, 2, 53], 'N': [7, 87, 113, 152, 154, 163, 164], 'P': [6, 43, 49, 55, 59, 135, 150, 158, 161, 177], 'PPII-helix': [9, 10, 11, 17, 53, 54, 159, 160, 161, 162, 171, 172, 173], 'Q': [8, 30, 45, 86, 88, 131, 172], 'R': [10, 14, 20, 27, 41, 65, 79, 90, 93, 116, 118, 181], 'S': [28, 38, 39, 40, 111, 149, 155, 159, 174, 178, 179], 'T': [9, 11, 94, 100, 102, 156, 169, 173], 'V': [34, 37, 48, 64, 67, 73, 96, 98, 101, 112, 123, 129, 136, 157, 168, 180], 'W': [46, 108, 120], 'Y': [12, 42, 68, 95, 97, 140], 'bend': [16, 27, 28, 40, 41, 45, 106, 107, 149, 153], 'turn': [6, 7, 14, 15, 29, 30, 43, 44, 55, 56, 87, 88, 89, 90, 132, 133, 148], 'unassigned': [4, 5, 8, 13, 31, 32, 39, 42, 48, 49, 57, 58, 72, 91, 105, 108, 115, 116, 117, 134, 150], 'ß-bridge': [12], 'ß-strand': [18, 19, 20, 21, 22, 23, 24, 25, 26, 33, 34, 35, 36, 37, 38, 46, 47, 50, 51, 52, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 118, 119, 120, 121, 122], 'α-helix': [59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 109, 110, 111, 112, 113, 114, 123, 124, 125, 126, 127, 128, 129, 130, 131, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147]}, 'A0A075B6H7':{'[1]Acetyl': [25, 27, 46, 48], '[7]Deamidated': [38, 44, 66], '[21]Phospho': [27], '[23]Dehydrated': [25, 27, 30, 46, 78], '[34]Methyl': [20, 21, 25, 26, 27, 30, 32, 34, 37, 46, 48, 50, 51, 52, 54, 55], '[35]Oxidation': [28, 29, 53], '[36]Dimethyl': [20], '[39]Methylthio': [43], '[47]Palmitoyl': [25, 27, 46, 48, 50, 51, 52, 55], '[142]HexNAc(1)dHex(1)': [48, 50, 51, 52, 55], '[143]HexNAc(2)': [25, 27], '[144]Hex(3)': [50, 51, 52, 55], '[148]Hex(1)HexNAc(2)': [25, 27], '[156]Hex(1)HexNAc(2)dHex(2)': [25, 27, 30], '[275]Nitrosyl': [43], '[299]Carboxy': [21], '[400]Tyr->Dha': [53], '[401]Didehydro': [27], '[405]Phosphoadenosine': [25, 27, 30], '[408]Glycosyl': [29], '[425]Dioxidation': [24, 28, 29], '[428]PhosphoHexNAc': [25, 30], '[429]PhosphoHex': [25], '[431]Palmitoleyl': [25, 42, 43, 46, 48, 50, 51, 52, 55], '[445]Hydroxytrimethyl': [60], '[449]Decanoyl': [25, 27], '[490]Hep': [25, 26, 27], '[528]Methyl+Deamidated': [26], '[529]Delta:H(5)C(2)': [28, 29], '[885]Label:13C(1)2H(3)+Oxidation': [24], '[1249]AHA-SS': [24], '[1250]AHA-SS_CAM': [24], '[1355]azole': [27, 32, 46, 48, 50, 51, 52], '[1413]PhosphoHex(2)': [25, 30], '[1425]Pentose': [30], '[1426]Hex(1)Pent(1)': [30], '[1433]HexNAc(3)': [25, 27, 30], '[1434]HexNAc(1)NeuAc(1)': [46, 48, 50, 51, 52], '[1441]Hex(1)Pent(3)': [25, 27], '[1442]Hex(1)NeuAc(1)Pent(1)': [25, 27], '[1445]dHex(2)Hex(2)': [25, 30], '[1600]Hex(1)HexNAc(2)NeuAc(1)': [25, 27], '[1602]Hex(1)HexNAc(2)NeuGc(1)': [25, 27], '[1604]Hex(5)Phos(1)': [25, 27], '[1612]dHex(2)Hex(4)': [25, 27, 30], '[1614]dHex(2)HexNAc(2)Kdn(1)': [25, 27, 40, 42, 46, 48, 50, 51, 52], '[1616]dHex(1)HexNAc(4)': [25, 27, 30], '[1668]dHex(2)Hex(3)HexNAc(2)Sulf(1)': [27], '[1746]dHex(2)Hex(3)HexNAc(5)': [25, 27, 30], '[1749]Hex(2)HexNAc(3)NeuAc(3)': [25, 27], '[1777]Hex(4)HexNAc(4)NeuAc(1)': [25, 27, 30], '[1913]glyoxalAGE': [38], '[1932]Hex(2)Sulf(1)': [25, 27, 30], '[1934]HexNAc(2)Sulf(1)': [50, 51, 52, 55], '[1935]Hex(1)Pent(3)Me(1)': [25, 27, 30], '[1936]Hex(2)Pent(2)': [25, 27, 30], '[1937]Hex(2)Pent(2)Me(1)': [25, 27, 30], '[1959]Hex(4)HexNAc(4)NeuGc(1)': [25, 27, 30], '[1960]dHex(4)Hex(3)HexNAc(2)NeuAc(1)': [25, 27, 30], 'IPR003599': [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115], 'IPR007110': [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116], 'IPR013106': [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111], '3₁₀-helix': [51, 52, 53, 101, 102, 103], 'A': [3, 5, 45, 64, 72, 76, 81, 105], 'C': [43, 109], 'D': [17, 91, 103, 112], 'E': [2, 21, 37, 102], 'F': [9, 83, 92, 104], 'G': [36, 62, 71, 85, 87, 89], 'I': [22, 69, 79, 96], 'IDR': [1, 2, 3, 4, 16, 17, 18], 'K': [60], 'L': [7, 8, 10, 11, 12, 13, 15, 31, 33, 41, 54, 67, 68, 94, 99, 115], 'M': [1, 24], 'N': [114], 'P': [4, 16, 28, 29, 35, 61, 65, 80, 101, 116], 'PPII-helix': [37, 38, 76], 'Q': [6, 26, 47, 58, 59, 63, 100, 110, 111], 'R': [20, 38, 44, 66, 75, 82], 'S': [27, 32, 34, 42, 46, 48, 50, 51, 52, 73, 78, 84, 86, 88, 97, 98], 'T': [18, 19, 25, 30, 40, 55, 74, 77, 90, 93, 95], 'V': [23, 39, 49, 106], 'W': [14, 56], 'Y': [53, 57, 70, 107, 108, 113], 'bend': [15, 29, 63, 97, 98, 105, 113], 'turn': [14, 35, 36, 61, 62, 71, 72, 73, 77, 78, 81, 82], 'unassigned': [19, 20, 21, 22, 23, 28, 30, 31, 32, 33, 34, 50, 60, 64, 65, 79, 80, 99, 100, 104, 112, 114, 115, 116], 'ß-strand': [24, 25, 26, 27, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 54, 55, 56, 57, 58, 59, 66, 67, 68, 69, 70, 74, 75, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 106, 107, 108, 109, 110, 111], 'α-helix': [5, 6, 7, 8, 9, 10, 11, 12, 13]}}

In [ ]:
# run on servers
import pandas as pd
import json

records = []
with open('protein_id_annotation_position.json', 'r') as f:
    data = json.load(f)

for protein_id, annotations in data.items():
    for annotation, positions in annotations.items():
        for pos in positions:
            records.append({
                'protein_id': protein_id,
                'position': pos,
                'annotation': annotation
            })

protmodcon = pd.DataFrame(records)
protmodcon.to_csv('protmodcon.csv', index=False)

In [10]:
protmodcon.head()

,protein_id,position,annotation
0,A0A024RBG1,18,IPR000086
1,A0A024RBG1,19,IPR000086
2,A0A024RBG1,20,IPR000086
3,A0A024RBG1,21,IPR000086
4,A0A024RBG1,22,IPR000086


In [ ]:
protmodcon['protein_pos'] = protmodcon['protein_id'].astype(str) + '_' + protmodcon['position'].astype(str)

In [72]:
if filters:
    mask = protmodcon.isin(filters).any(axis=1)
    protmodcon = protmodcon[mask]
    
# compute pairwise overlaps across all columns
overlap_dict = {}

for x in x_types:
    mask_x = protmodcon.isin([x]).any(axis=1)
    x_set = set(
        protmodcon[mask_x]['protein_id'].astype(str) + '_' + protmodcon[mask_x]['position'].astype(str)
    )
    for y in y_types:
        mask_y = protmodcon.isin([y]).any(axis=1)
        y_set = set(
            protmodcon[mask_y]['protein_id'].astype(str) + '_' + protmodcon[mask_y]['position'].astype(str)
        )
        overlap_dict[(x, y)] = len(x_set & y_set)

print(overlap_dict)

{('[21]Phospho', 'ß-strand'): 0, ('[21]Phospho', 'K'): 4, ('[1]Acetyl', 'ß-strand'): 2, ('[1]Acetyl', 'K'): 5}
